In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture --no-stderr
!pip install python-dotenv langchain_openai langchain-chroma pypdf langchain langchain_community

In [ ]:
# 환경변수 설정

In [3]:
# 라이브러리 불러오기
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/drive/MyDrive/Colab Notebooks/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# 예제 2-18 <2024 부동산 보고서 RAG 챗봇>
# 라이브러리 불러오기
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

In [5]:
# PDF 문서 로드 및 텍스트 분할
loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/langchain_book/2024_KB_부동산_보고서_최종.pdf")
documents = loader.load()  # 문서 로드

# 텍스트 분할 설정: 청크 크기와 겹침 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

# 분할된 청크 수
print('분할된 청크의 수:', len(chunks))

분할된 청크의 수: 138


In [6]:
# 임베딩 생성 및 Chroma 데이터베이스 저장
embedding_function = OpenAIEmbeddings()  # 임베딩 모델 설정

persist_directory = "/content/drive/MyDrive/Colab Notebooks/langchain_book/directory/chroma"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    persist_directory=persist_directory  # 데이터베이스 저장 경로
)

In [7]:
print('문서의 수:', vectorstore._collection.count())

문서의 수: 276


In [8]:
# 검색 및 재정렬
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})  # 관련 문서 상위 3개 검색 설정

# 프롬프트 템플릿 설정: 사용자 질문에 대한 답변을 생성하기 위한 템플릿
template = """당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.

컨텍스트: {context}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("placeholder", "{chat_history}"),
        ("human", "{question}")
    ]
)  # 템플릿 초기화
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # AI 모델 설정

In [9]:
print(prompt.format(context="컨텍스트 예시", chat_history=["대화 기록 예시1", "대화 기록 예시2"], question="질문 예시"))

System: 당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.

컨텍스트: 컨텍스트 예시

Human: 대화 기록 예시1
Human: 대화 기록 예시2
Human: 질문 예시


In [10]:
# 문서 형식 변환 함수 정의
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)  # 문서 내용을 줄바꿈으로 연결

In [11]:
# 체인 구성: 검색한 문서를 프롬프트에 연결하고 모델을 통해 응답 생성
chain = (
    RunnablePassthrough.assign(
        context=lambda x: format_docs(retriever.invoke(x["question"]))
    )  # 검색된 문서를 연결하여 전달
    | prompt
    | model
    | StrOutputParser()  # 결과를 문자열로 변환
)

In [12]:
# 대화 기록을 유지하기 위한 메모리 설정
chat_history = ChatMessageHistory()
chain_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,  # 세션 ID별 대화 기록 생성
    input_messages_key="question",
    history_messages_key="chat_history",
)

In [13]:
# 챗봇 실행 함수 정의
def chat_with_bot():
    session_id = "user_session"
    print("KB 부동산 보고서 챗봇입니다. 질문해 주세요. (종료하려면 'quit' 입력)")
    while True:
        user_input = input("사용자: ")
        if user_input.lower() == 'quit':
            break

        response = chain_with_memory.invoke(
            {"question": user_input},
            {"configurable": {"session_id": session_id}}
        )

        print("챗봇:", response)

# 메인 실행
if __name__ == "__main__":
    chat_with_bot()

KB 부동산 보고서 챗봇입니다. 질문해 주세요. (종료하려면 'quit' 입력)
사용자: 안녕?
챗봇: 안녕하세요! 궁금하신 내용이 있으신가요? 부동산 관련해서 궁금한 점이 있으시면 언제든지 물어보세요. 도움이 될 수 있도록 최선을 다해 도와드리겠습니다.
사용자: 2024년도 부도산 집값
챗봇: 2024년에 대한 정확한 부동산 집값 정보는 현재로서는 제공할 수 없습니다. 부동산 시장은 다양한 요인에 영향을 받기 때문에 미래의 집값을 정확히 예측하기는 어렵습니다. 

그러나 2024년에는 주택 시장에서 주택담보대출 증가세가 다소 둔화될 것으로 전망되고, 주택공급 부족이 여전히 주요 이슈로 남을 것으로 예상됩니다. 이러한 요인들이 부동산 집값에 영향을 줄 수 있습니다.

더 자세한 정보나 예측을 원하신다면 해당 시기에 가장 가까운 시점에 발표된 부동산 시장 보고서나 전문가의 의견을 참고하시는 것이 좋을 것입니다. 부동산 시장은 변화무쌍하니까요. 추가로 궁금한 점이 있으시면 언제든지 물어보세요. 도움이 될 수 있도록 최선을 다해 도와드리겠습니다.
사용자: quit
